In [ ]:
# imports
import requests
import os
import json
import pandas as pd

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
key = os.environ["FOURSQUARE_KEY"]
headers = {
    "accept": "application/json",
    "Authorization": key
}

foursquare_poi = {}

for station in stations[:250]: # Using 250 stations for processing time
    url = "https://api.foursquare.com/v3/places/search?ll=" + str(station["latitude"]) + "%2C" + str(station["longitude"]) + "&radius=1000&fields=name,categories,location,distance,rating&sort=RELEVANCE"
    response = json.loads(requests.get(url, headers=headers).text)
    foursquare_poi[station["station_name"]] = response

print(foursquare_poi)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
clean_foursquare_poi = []

for station in foursquare_poi:
    for poi in foursquare_poi[station]["results"]:
        clean_foursquare_poi.append(
            {
                "station_name": station,
                "station_distance_(m)": poi["distance"],
                "loc_address": poi["location"]["formatted_address"],
                "loc_name": poi["name"],
                "loc_category": poi["categories"][0]["name"],
                "loc_rating": poi.get("rating", None)
            }
        )

Put your parsed results into a DataFrame

In [ ]:
foursquare_df = pd.DataFrame(clean_foursquare_poi)
foursquare_df

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
key = os.environ["YELP_KEY"]
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {key}"
}

yelp_poi = {}

for station in stations[:250]:
    url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(station["latitude"]) + "&longitude=" + str(station["longitude"]) + "&radius=1000&categories=&sort_by=best_match&limit=10" # Match Foursquare default limit of 10
    response = json.loads(requests.get(url, headers=headers).text)
    yelp_poi[station["station_name"]] = response

print(yelp_poi)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
clean_yelp_poi = []

for station in yelp_poi:
    for poi in yelp_poi[station]["businesses"]:
        clean_yelp_poi.append(
            {
                "station_name": station,
                "station_distance_(m)": round(poi["distance"]),
                "loc_address": poi["location"]["address1"],
                "loc_name": poi["name"],
                "loc_category": poi["categories"][0]["title"],
                "loc_rating": poi["rating"]
            }
        )

Put your parsed results into a DataFrame

In [ ]:
yelp_df = pd.DataFrame(clean_yelp_poi)
yelp_df

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

'''

Both APIs actually provided enough data for the information relevant to what I am comparing. By using the API parameters and field filtering, I can find the name, location, rating, etc. for all POIs. However, the rating scale provided by the APIs is different; Foursquare provides ratings on a scale /10, whereas Yelp uses a rating scale /5. This will make it difficult to compare the ratings of POIs that are sourced from different APIs.

'''

Get the top 10 restaurants according to their rating

In [ ]:
foursquare_restaurants = foursquare_df.loc[foursquare_df["loc_category"] == "Restaurant"]
foursquare_restaurants = foursquare_restaurants.sort_values(by="loc_rating", ascending=False)
foursquare_restaurants.head(10)

In [ ]:
restaurants = ["Restaurant", "Chinese", "Lebanese", "Vietnamese", "Italian", "British", "Pizza", "Turkish", "Sandwiches"] # The Yelp API doesn't categorize restaurants as a general term

yelp_restaurants = yelp_df.loc[yelp_df["loc_category"].isin(restaurants)]
yelp_restaurants = yelp_restaurants.sort_values(by="loc_rating", ascending=False)
yelp_restaurants.head(10)